<left> <img src="https://radicalbit.ai/wp-content/uploads/2024/02/radicalbit-logo-bk.png" width="400" /> </left>


## Radicalbit Quickstart: Monitor a Binary Classification Model

### Introduction
This guide provides instructions on how to monitor a ML solution with the Radicalbit OS Platform, through the Python SDK (https://pypi.org/project/radicalbit-platform-sdk/).



In [42]:
from radicalbit_platform_sdk.client import Client
from radicalbit_platform_sdk.models import (
    AwsCredentials,
    CreateModel,
    DataType,
    ModelType,
    ColumnDefinition,
    OutputType,
    Granularity,
)

from datetime import datetime 
import pandas as pd



### Create the Client
In order to communicate with the platform, you need to create the client and indicate the URL where the UI will be available.
Remember that before you need to launch the platform following the instructions in the README.md (https://github.com/radicalbit/radicalbit-ai-monitoring/blob/main/README.md).

In [43]:
# Create the Client
base_url = "http://localhost:9000"
client = Client(base_url)

### The reference dataset
The reference dataset is the name we use to indicate the batch that contains the information we desire to have constantly (or we expect to have) over time. It could be the training set or a chunck of production data where the model has had good performances.

To use the radicalbit-ai-monitoring platform, you need first to prepare your reference data, which should include the following information:

- **Variables**: The list of features used by the model as well as other information like metadata produced by the system
- **Outputs**: The fields returned by the model after the inference. Usually, they are probabilities, a predicted class or numbers.
- **Target**: the ground truth used to validate predictions and evaluate the model quality
- **Timestamp**: The timestamp field used to aggregate data over selected windows.

In this example we will use a dataset build to classify between two different classes:
- **class 0:** income <= 50K
- **class 1:** income > 50K


> **_Dataset license:_**  Kohavi,Ron. (1996). Census Income. UCI Machine Learning Repository. https://doi.org/10.24432/C5GP7S. Adapted by Radicalbit.

In [34]:
reference = pd.read_csv("../data/binary_classification/reference-income.csv")
reference.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income_>50K,pred_id,prob_class_1,prediction,timestamp
0,37,Private,174844,Some-college,10,Married-civ-spouse,Other-service,Husband,White,Male,0,0,17,United-States,0,fe2f8443-b644-4ba5-a130-45336a0a04d6,0.179987,0,2024-01-09 23:59:00
1,42,NaN,167678,11th,7,Married-civ-spouse,NaN,Husband,White,Male,0,0,22,Ecuador,0,b85abaca-581e-4988-8392-03460b3aa5d5,0.126728,0,2024-01-10 00:08:00
2,27,Self-emp-not-inc,115438,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,60,Ireland,1,01d05d15-8d34-4451-97d5-99dff5ee9217,0.248176,0,2024-01-10 00:21:00
3,32,Private,295589,Masters,14,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,1977,40,United-States,1,01a9168f-fbb9-46e4-acfd-c1be174a9373,0.853719,1,2024-01-10 00:33:00
4,20,Private,107746,11th,7,Never-married,Transport-moving,Other-relative,White,Male,0,0,40,Guatemala,0,3a6074d2-b42f-4853-aeac-d164d36baad2,0.012086,0,2024-01-10 00:42:00


### Create the Model
The next step requires the Model creation.
Here, you need to specify the following information:

- **name:** The name of the model
- **model_type:** The type of the model
- **data_type:** It explains the data type used by the model
- **granularity:** The window used to calculate aggregated metrics with the current data
- **features:** A list column representing the features set
- **outputs:** An OutputType definition to explain the output of the model
- **target:** The column used to represent the model's target
- **timestamp:** The column used to store the when prediction was done
- **frameworks:** An optional field to describe the frameworks used by the model
- **algorithm:** An optional field to explain the algorithm used by the model

In [36]:
# Create the Model
model = CreateModel(
    name=f"Linear-Regressor-{str(datetime.now()).replace(" ", "-").replace(":", "-").split(".")[0]}",
    modelType=ModelType.BINARY,
    dataType=DataType.TABULAR,
    granularity=Granularity.HOUR,
    features=[
        ColumnDefinition(name="age", type="int"),
        ColumnDefinition(name="workclass", type="string"),
        ColumnDefinition(name="fnlwgt", type="int"),
        ColumnDefinition(name="education", type="string"),
        ColumnDefinition(name="educational-num", type="int"),
        ColumnDefinition(name="marital-status", type="string"),
        ColumnDefinition(name="occupation", type="string"),
        ColumnDefinition(name="relationship", type="string"),
        ColumnDefinition(name="race", type="string"),
        ColumnDefinition(name="gender", type="string"),
        ColumnDefinition(name="capital-gain", type="int"),
        ColumnDefinition(name="capital-loss", type="int"),
        ColumnDefinition(name="hours-per-week", type="int"),
        ColumnDefinition(name="native-country", type="string"),
    ],
    outputs=OutputType(
        prediction=ColumnDefinition(name="prediction", type="int"),
        output=[
            ColumnDefinition(name="prob_class_1", type="float"),
            ColumnDefinition(name="prediction", type="int")
        ],
    ),
    target=ColumnDefinition(name="income_>50K", type="int"),
    timestamp=ColumnDefinition(name="timestamp", type="datetime"),
)

model = client.create_model(model)
print(model.name())
print(model.uuid())


Linear-Regressor-2024-07-01-14-44-47
b17ff651-a66a-4f4b-bd6e-0d503d31d859


After this action, go to the platform to see:
- In the **Overview** section, you will see the generated schema of Variables and Outputs


### Load the reference dataset
Once the model has been created, you are ready to upload your reference dataset into the platform. All you need is to run the following code, in which you have to specify the path of your file and set up your AWS credentials as indicated here. 
In this case, we use Minio as a substitute for a real AWS. 



In [37]:
# load the reference dataset
ref = model.load_reference_dataset(
    file_name="../data/binary_classification/reference-income.csv",
    bucket="test-bucket",
    aws_credentials=AwsCredentials(
        access_key_id="minio",
       secret_access_key="minio123",
       default_region="us-east-1",
        endpoint_url="http://localhost:9090"
    )
)

print(ref.uuid())

After this action, go to the platform to see:
- In the **Overview/Summary** section, you will see a summary of your data (missing values, number of rows or columuns and other)y
- in the **Reference** section you will see information about Data Quality and Model Quality

### Load the current dataset
The last step regards the current data uploading. The current dataset is the name we use to indicate the batch that contains fresh information, for example, the most recent production data, predictions or ground truths. We expect that it has the same characteristics (statistical properties) as the reference, which indicates that the model has the performance we expect and there is no drift in the data.
As you can see, the code is pretty similar to the reference one. 

In [40]:
# load the current dataset
cur1 = model.load_current_dataset(
    file_name="../data/binary_classification/current1-income.csv",
    correlation_id_column="pred_id",
    bucket="test-bucket",
    aws_credentials=AwsCredentials(
        access_key_id="minio",
       secret_access_key="minio123",
       default_region="us-east-1",
        endpoint_url="http://localhost:9090"
    )
)

After this action, go to the platform to see:
- in the **Current** section you will see information about Data Quality and Model Quality compared to the Reference information
- in the **Current/Import** section you will see and browse your uploaded current data